In [1]:
import numpy as np
import sys

import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append("/Users/clemens/Documents/Code/AnalysisToolbox")

import spikeAnalysisToolsV2.data_loading as data
import spikeAnalysisToolsV2.firing_rates as firing
import spikeAnalysisToolsV2.helper as helper
import spikeAnalysisToolsV2.overviews as overview
import spikeAnalysisToolsV2.information_scores as info
import spikeAnalysisToolsV2.plotting as spikeplot
import spikeAnalysisToolsV2.combine_stimuli as combine
import spikeAnalysisToolsV2.synapse_analysis as synapse_analysis

In [3]:
## set the Masterpath to the folder where your output is saved

masterpath = "/Users/clemens/Documents/Code/ModelClemens/output"
## set the subfolder to the Simulation you want to analyse 

subfolders = [
    "12_10-21_59_all_CT"
            ]
## if more than the inital epoch is needed *1 needs to be run
extensions = ["initial"] + data.get_epochs(masterpath+"/"+subfolders[0])[::3]

object_list_old = data.load_testing_stimuli_info(masterpath+ "/" + subfolders[0]) #assuming all the subfolders have the same
n_stimuli = np.sum(obj['count'] for obj in object_list_old)


# info_neurons is just an array of the information from above. This makes it easier to run the functions and pass the information. 
# info_times same for times
network_architecture = dict(
    num_exc_neurons_per_layer = 64*64,
    num_inh_neurons_per_layer = 32*32,
    num_layers = 4
)

info_times = dict(
    length_of_stimulus = 2.0,
    num_stimuli = n_stimuli,
    time_start = 1.5,
    time_end = 1.9
)


In [4]:
object_list_shape = data.load_testing_stimuli_indices_from_wildcarts(masterpath+"/" + subfolders[0], ["***r", "***l"])
object_indices_shape = [o['indices'] for o in object_list_shape]

object_indices = object_indices_shape

In [ ]:
spikes = data.load_spikes_from_subfolders(masterpath, subfolders, extensions, False)
print("loading spikes done")

rates_subfolders = firing.calculate_rates_subfolder(
    spikes,
    network_architecture, 
    info_times)
spikes = None

exc_rates, inh_rates = helper.nested_list_of_epochs_2_np(rates_subfolders[0])

exc_info, inh_info = info.information_all_epochs(rates_subfolders[0], 'mutual_info', objects=object_indices, n_bins=3)
rates_subfolders = None
print(exc_info.shape)

exc_rates_objects = combine.average_responses(exc_rates, object_indices, 1)
inh_rates_objects = combine.average_responses(inh_rates, object_indices, 1)

exc_rates = None
inh_rates = None

# Network Architecture
net, weights = data.load_weights_all_epochs(masterpath+"/"+subfolders[0], extensions[1:])
mask = synapse_analysis.Synapse_Mask(network_architecture, net)

print(".../done")

Start


In [ ]:
spikeplot.plot_information_development(exc_info, mean_of_top_n=1000, threshold=0.9)

# Development of Average Firing rates
### Overall Average Firing Rate within Layer

In [ ]:
print(exc_rates_objects.shape)
spikeplot.plot_development_for_object(np.mean(exc_rates_objects, axis=3), "Excitatory Average FR")
spikeplot.plot_development_for_object(np.mean(inh_rates_objects, axis=3), "Inhibitory Average FR")

### Average Firing Rate For Informative neurons

In [ ]:
neuron_informative = np.any(exc_info > 0.5, axis=0)
print(exc_rates_objects.shape)


mean_informative_neurons = np.empty(exc_rates_objects.shape[:3])

for l in range(mean_informative_neurons.shape[2]): # go threw layers
    mean_informative_neurons[:, :, l] = np.mean(exc_rates_objects[:, :, l, neuron_informative[0, l, :]], axis=2)

# Neurons that at any time had information >0.5
# Has shape layer, neuron
#informative_layers, informative_neurons = np.where(neuron_informative)

#print(exc_rates_objects)
#print(exc_rates_objects[:, :, informative_layers, infor].shape)
mean_informative_neurons[np.isnan(mean_informative_neurons)] = 0

spikeplot.plot_development_for_object(mean_informative_neurons, "Excitatory Average FR for informative neurons")

In [ ]:
neuron_not_informative = np.invert(neuron_informative)
print(exc_rates_objects.shape)


mean_not_informative_neurons = np.empty(exc_rates_objects.shape[:3])

for l in range(mean_not_informative_neurons.shape[2]): # go threw layers
    mean_not_informative_neurons[:, :, l] = np.mean(exc_rates_objects[:, :, l, neuron_not_informative[0, l, :]], axis=2)

spikeplot.plot_development_for_object(mean_not_informative_neurons, "Excitatory Average FR for UN-informative neurons")

# Finding interesting neurons

In [ ]:
layer = 3
epoch_in_which_to_max = 70

decreasing_indices = np.argsort(exc_info, axis=3)[:,:, :, ::-1]
target_n = decreasing_indices[epoch_in_which_to_max, 0, layer, 0]


# Information vs Firing Rate

In [ ]:
# info all neurons
# firing rate for all objects and all neurons plot them
# maybe with sum of incoming weights
# maybe with animated historgram of incomming weights.

In [ ]:
n_obj = exc_rates_objects.shape[1]
def plot_neuron(pos, rate_average=True):
    layer, neuron_id = pos
    
    fig = plt.figure(figsize=(15,12))
    ax = fig.add_subplot(1,2,1)
    ax.set_title("Neuron {}".format(pos))
    
    overall_id = helper.position_to_id(pos, True, network_architecture)
    
    incoming_exc_mask = np.logical_and(np.invert(mask.inh_lateral()), (net.post.values == overall_id) )
    incoming_inh_mask = np.logical_and(mask.inh_lateral(), net.post.values == overall_id)
    incoming_weights = weights[:, incoming_exc_mask]
    mean_w = np.mean(incoming_weights, axis=1)
    
    # plot information dev
    ax.plot(exc_info[:, 0, layer, neuron_id], label="information development")
    
    # weight dev
    ax.plot(mean_w, label="Mean incomming exc weights")
    
    
    if rate_average:
        # average rates of this neuron
        rates_neuron = exc_rates_objects[:, :, layer, neuron_id]
        max_rate_neuron = np.max(rates_neuron)
        rates_neuron /= max_rate_neuron
    
        for obj in range(n_obj):
            ax.plot(rates_neuron[:, obj], label="FR to obj {} /={:.2f}".format(obj, max_rate_neuron))
    else:
        # scatter the indivividual firing rates for each presentation
        n_epochs = exc_info.shape[0]
        
        max_rate = np.max(exc_rates[:, :, layer, neuron_id])
        
        for i, obj in enumerate(object_indices):
            firing_rates = exc_rates[:, obj, layer, neuron_id].flatten('C')/max_rate
            epoch_numbers = np.repeat(np.arange(n_epochs), len(obj))
            #print(epoch_numbers.shape)
            #print(firing_rates.shape)
            plt.scatter(epoch_numbers, firing_rates, marker='.', label="FR to obj {} /= {:.2f}".format(i, max_rate))    
    ax.legend()
    
    ax2 = fig.add_subplot(1,2,2)
    ax2.set_title("Neuron {} - Currents".format(pos))
    print(weights.shape)
    print(exc_rates_objects.shape)
    exc_current = synapse_analysis.weighted_presynaptic_actvity(incoming_exc_mask, net, weights, (exc_rates_objects, inh_rates_objects))
    inh_current = synapse_analysis.weighted_presynaptic_actvity(incoming_inh_mask, net, weights, (exc_rates_objects, inh_rates_objects))
    
    obj=0
    ax2.plot(exc_current[:, obj], color='C2', label="Excitatory obj {}".format(obj))
    ax2.plot(-inh_current[:, obj], color='green', label="Inhibitory obj {}".format(obj))
    obj=1
    ax2.plot(exc_current[:, obj], color='C3', label="Excitatory obj {}".format(obj))
    ax2.plot(-inh_current[:, obj], color='red', label="Inhibitory obj {}".format(obj))
        
    ax2.legend()
    
    

In [ ]:
target_n = decreasing_indices[epoch_in_which_to_max, 0, layer, 0]


# Nice Neuron: L 2, N 1428
neuron_pos = (layer, target_n)
plot_neuron(neuron_pos, True)

In [ ]:
target_n = decreasing_indices[epoch_in_which_to_max, 0, layer, 1]


# Nice Neuron: L 2, N 1428
neuron_pos = (layer, target_n)
plot_neuron(neuron_pos, True)

In [ ]:
target_n = decreasing_indices[epoch_in_which_to_max, 0, layer, 2]


# Nice Neuron: L 2, N 1428
neuron_pos = (layer, target_n)
plot_neuron(neuron_pos, True)

In [ ]:
target_n = decreasing_indices[epoch_in_which_to_max, 0, layer, 3]


# Nice Neuron: L 2, N 1428
neuron_pos = (layer, target_n)
plot_neuron(neuron_pos, True)

In [ ]:
target_n = decreasing_indices[epoch_in_which_to_max, 0, layer, 4]


# Nice Neuron: L 2, N 1428
neuron_pos = (layer, target_n)
plot_neuron(neuron_pos, True)

In [ ]:
target_n = decreasing_indices[epoch_in_which_to_max, 0, layer, 5]


# Nice Neuron: L 2, N 1428
neuron_pos = (layer, target_n)
plot_neuron(neuron_pos, True)

In [ ]:
target_n = decreasing_indices[epoch_in_which_to_max, 0, layer, 70]


# Nice Neuron: L 2, N 1428
neuron_pos = (layer, target_n)
plot_neuron(neuron_pos)